In [25]:
from transformers import pipeline,AutoTokenizer, AutoModel
import random
from morfeusz2 import Morfeusz

In [26]:
morf = Morfeusz()

In [3]:
random.seed(420)

# Download three Polish models from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples.



In [12]:
papuga = pipeline('text-generation', model='flax-community/papuGaPT2-large')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
herbert = pipeline(task="fill-mask", model='allegro/herbert-base-cased')

In [13]:
xlm = pipeline('fill-mask', model='facebook/xlm-v-base')

In [125]:
bart = pipeline("fill-mask", model="sdadas/polish-bart-base")

# Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [16]:
sentence_papuga='Wczoraj wieczorem poszedłem do <mask>.'

In [17]:
papuga(sentence_papuga)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Wczoraj wieczorem poszedłem do <mask>. Na drugi dzień po prostu wybiegłam na zewnątrz i przeniosłam się na drugą stronę miasta (teraz już w zasadzie o tej samej porze). Przeniosłam się na drugą stronę, bo nie planowałam wejść na'}]

In [20]:
sentence2="Warszawa to największe <mask>."

In [24]:
xlm(sentence2)

[{'score': 0.893892765045166,
  'token': 11381,
  'token_str': 'miasto',
  'sequence': 'Warszawa to największe miasto.'},
 {'score': 0.02109367586672306,
  'token': 4122,
  'token_str': 'miasta',
  'sequence': 'Warszawa to największe miasta.'},
 {'score': 0.011969583109021187,
  'token': 3610,
  'token_str': 'centrum',
  'sequence': 'Warszawa to największe centrum.'},
 {'score': 0.008560607209801674,
  'token': 154945,
  'token_str': 'lotnisko',
  'sequence': 'Warszawa to największe lotnisko.'},
 {'score': 0.005978821311146021,
  'token': 2104,
  'token_str': 'miejsce',
  'sequence': 'Warszawa to największe miejsce.'}]

In [22]:
herbert(sentence2)

[{'score': 0.8103944063186646,
  'token': 4618,
  'token_str': 'miasto',
  'sequence': 'Warszawa to największe miasto.'},
 {'score': 0.082491934299469,
  'token': 23562,
  'token_str': 'lotnisko',
  'sequence': 'Warszawa to największe lotnisko.'},
 {'score': 0.0265207439661026,
  'token': 4831,
  'token_str': 'centrum',
  'sequence': 'Warszawa to największe centrum.'},
 {'score': 0.01798691414296627,
  'token': 3073,
  'token_str': 'miasta',
  'sequence': 'Warszawa to największe miasta.'},
 {'score': 0.005562868434935808,
  'token': 21445,
  'token_str': 'atrakcje',
  'sequence': 'Warszawa to największe atrakcje.'}]

In [123]:
def analyze_correctness(word):
    return set([z
                    for x in morf.analyse(word)
                    if x[2][2].split(':')[0] == 'subst'
                    for z in x[2][2].split(':')[2].split('.')])

In [124]:
analyze_correctness('mleko')

{'acc', 'nom', 'voc'}

In [94]:
polish_test = {
    'mianownik': ('Bartek poszedł kupić <mask>.', 'nom', 'mleko'),
    'dopełniacz': ('Basia nie kupiła <mask>.', 'gen', 'mleka'),
    'celownik': ('Uczeń przygląda się pani <mask>.', 'dat', 'nauczycielce'),
    'biernik': ('Mama gotuje <mask>.', 'acc', 'obiad'),
    'narzędnik': ('Natalia interesuje się <mask>.', 'inst', 'malarstwem'),
    'miejscownik': ('Myślę o wczorajszym <mask> i ciągle mam na niego ochotę.', 'loc', 'deserze'),
    'wołacz': ('Przybywasz za późno, <mask>.', 'voc', 'bohaterze')
}

In [135]:
points_herb=0
for text in polish_test:
    print("#########")
    print(polish_test[text])
    print("#########")
    expected=polish_test[text][1]
    pred=herbert(polish_test[text][0])
    for sentence in pred:
        print(sentence['sequence'])
        if expected in analyze_correctness(sentence['token_str']):
            print('correct form')
            print("--------")
            points_herb+=1

    print("===============================================")

print(f"Result: {points_herb}/35")

#########
('Bartek poszedł kupić <mask>.', 'nom', 'mleko')
#########
Bartek poszedł kupić buty.
correct form
--------
Bartek poszedł kupić rower.
correct form
--------
Bartek poszedł kupić telewizor.
correct form
--------
Bartek poszedł kupić książki.
correct form
--------
Bartek poszedł kupić książkę.
#########
('Basia nie kupiła <mask>.', 'gen', 'mleka')
#########
Basia nie kupiła książki.
correct form
--------
Basia nie kupiła samochodu.
correct form
--------
Basia nie kupiła mieszkania.
correct form
--------
Basia nie kupiła auta.
correct form
--------
Basia nie kupiła butów.
correct form
--------
#########
('Uczeń przygląda się pani <mask>.', 'dat', 'nauczycielce')
#########
Uczeń przygląda się pani dyrektor.
correct form
--------
Uczeń przygląda się pani doktor.
correct form
--------
Uczeń przygląda się pani profesor.
correct form
--------
Uczeń przygląda się pani domu.
Uczeń przygląda się pani Dyrektor.
correct form
--------
#########
('Mama gotuje <mask>.', 'acc', 'obiad')
####

In [136]:
points_xlm=0
for text in polish_test:
    print("#########")
    print(polish_test[text])
    print("#########")
    expected=polish_test[text][1]
    pred=xlm(polish_test[text][0])
    for sentence in pred:
        print(sentence['sequence'])
#         print(get_predicted_declension(sentence['token_str']))
        if expected in get_predicted_declension(sentence['token_str']):
            print('correct form')
            print("--------")
            points_xlm+=1

    print("===============================================")

print(f"Result: {points_xlm}/35")

#########
('Bartek poszedł kupić <mask>.', 'nom', 'mleko')
#########
Bartek poszedł kupić piwo.
correct form
--------
Bartek poszedł kupić książkę.
Bartek poszedł kupić prezenty.
correct form
--------
Bartek poszedł kupić śniadanie.
correct form
--------
Bartek poszedł kupić wino.
correct form
--------
#########
('Basia nie kupiła <mask>.', 'gen', 'mleka')
#########
Basia nie kupiła mnie.
Basia nie kupiła..
Basia nie kupiła nic.
correct form
--------
Basia nie kupiła sie.
Basia nie kupiła....
#########
('Uczeń przygląda się pani <mask>.', 'dat', 'nauczycielce')
#########
Uczeń przygląda się panii.
Uczeń przygląda się paniom.
Uczeń przygląda się pani..
Uczeń przygląda się pani domu.
Uczeń przygląda się panią.
#########
('Mama gotuje <mask>.', 'acc', 'obiad')
#########
Mama gotuje..
Mama gotuje:.
Mama gotuje :.
Mama gotuje,.
Mama gotuje....
#########
('Natalia interesuje się <mask>.', 'inst', 'malarstwem')
#########
Natalia interesuje się muzyką.
correct form
--------
Natalia interesuje 

In [137]:
points_bart=0
for text in polish_test:
    print("#########")
    print(polish_test[text])
    print("#########")
    expected=polish_test[text][1]
    pred=bart(polish_test[text][0])
    for sentence in pred:
        print(sentence['sequence'])
#         print(get_predicted_declension(sentence['token_str']))
        if expected in get_predicted_declension(sentence['token_str']):
            print('correct form')
            print("--------")
            points_bart+=1

    print("===============================================")

print(f"Result: {points_bart}/35")

#########
('Bartek poszedł kupić <mask>.', 'nom', 'mleko')
#########
Bartek poszedł kupić sobie.
Bartek poszedł kupić mu.
Bartek poszedł kupić coś.
correct form
--------
Bartek poszedł kupić książkę.
Bartek poszedł kupić bilet.
correct form
--------
#########
('Basia nie kupiła <mask>.', 'gen', 'mleka')
#########
Basia nie kupiła jeszcze.
Basia nie kupiła sobie.
Basia nie kupiła biletów.
correct form
--------
Basia nie kupiła już.
Basia nie kupiła mieszkania.
correct form
--------
#########
('Uczeń przygląda się pani <mask>.', 'dat', 'nauczycielce')
#########
Uczeń przygląda się panice.
correct form
--------
Uczeń przygląda się pani doktor.
correct form
--------
Uczeń przygląda się pani profesor.
correct form
--------
Uczeń przygląda się pani z.
Uczeń przygląda się pani w.
#########
('Mama gotuje <mask>.', 'acc', 'obiad')
#########
Mama gotuje..
Mama gotuje dla.
Mama gotuje i.
Mama gotuje z.
Mama gotuje,.
#########
('Natalia interesuje się <mask>.', 'inst', 'malarstwem')
#########
Nata

In [139]:
print(f"final_results:\nherbert: {points_herb}/35\nxlm: {points_xlm}/35\nbart: {points_bart}/35")

final_results:
herbert: 31/35
xlm: 16/35
bart: 11/35


Despite results, these models are able to recognize correct word to fill the mask. In most cases, proposed words make sense, but in model xlm in shorter sentences, proposed examples sometimes are not matching expected result.

# Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [152]:
import re
fill_sentence = lambda model, sentence: re.sub("<mask>", model(sentence)[0]['token_str'], sentence)

In [173]:
example_1="Nauczycielka <mask> swoich uczniów i nie chciała sie z nimi rozstawać."
example_2="Lekarka poznała mężczyznę, więc <mask> by się z nim zobaczyć"
example_3='Architekt uwielbiał swoją pracę, ale mimo to <mask> ją opuścić.'

In [164]:
fill_sentence(bart,example_1)

'Nauczycielka bardzo swoich uczniów i nie chciała sie z nimi rozstawać.'

In [165]:
fill_sentence(herbert,example_1)

'Nauczycielka lubiła swoich uczniów i nie chciała sie z nimi rozstawać.'

In [166]:
fill_sentence(xlm,example_1)

'Nauczycielka kocha swoich uczniów i nie chciała sie z nimi rozstawać.'

In [167]:
fill_sentence(bart,example_2)

'Lekarka poznała mężczyznę, więc nie by się z nim zobaczyć'

In [168]:
fill_sentence(herbert,example_2)

'Lekarka poznała mężczyznę, więc przyszła by się z nim zobaczyć'

In [169]:
fill_sentence(xlm,example_2)

'Lekarka poznała mężczyznę, więc chciała by się z nim zobaczyć'

In [174]:
fill_sentence(bart,example_3)

'Architekt uwielbiał swoją pracę, ale mimo to nie ją opuścić.'

In [175]:
fill_sentence(herbert,example_3)

'Architekt uwielbiał swoją pracę, ale mimo to musiał ją opuścić.'

In [176]:
fill_sentence(xlm,example_3)

'Architekt uwielbiał swoją pracę, ale mimo to postanowił ją opuścić.'

# Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [211]:
example_r_w_knowledge_1="<mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza."
example_r_w_knowledge_2="<mask>, czyli jedna z planet Układu Słonecznego, trzecia od Słońca; jest znana z bogactwa różnorodnych form życia."
example_r_w_knowledge_3="<mask> czyli proces, podczas którego substancje zmieniają się z ciekłego stanu w gazowy bez bezpośredniego przechodzenia przez stan stały."

In [180]:
fill_sentence(bart,example_r_w_knowledge_1)

'W wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.'

In [181]:
fill_sentence(herbert,example_r_w_knowledge_1)

'Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.'

In [182]:
fill_sentence(xlm,example_r_w_knowledge_1)

'Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.'

In [212]:
fill_sentence(bart,example_r_w_knowledge_2)

'Plane, czyli jedna z planet Układu Słonecznego, trzecia od Słońca; jest znana z bogactwa różnorodnych form życia.'

In [213]:
fill_sentence(herbert,example_r_w_knowledge_2)

'Wenus, czyli jedna z planet Układu Słonecznego, trzecia od Słońca; jest znana z bogactwa różnorodnych form życia.'

In [214]:
fill_sentence(xlm,example_r_w_knowledge_2)

'Pluto, czyli jedna z planet Układu Słonecznego, trzecia od Słońca; jest znana z bogactwa różnorodnych form życia.'

In [201]:
fill_sentence(bart,example_r_w_knowledge_3)

'Gaz czyli proces, podczas którego substancje zmieniają się z ciekłego stanu w gazowy bez bezpośredniego przechodzenia przez stan stały.'

In [202]:
fill_sentence(herbert,example_r_w_knowledge_3)

'Jest czyli proces, podczas którego substancje zmieniają się z ciekłego stanu w gazowy bez bezpośredniego przechodzenia przez stan stały.'

In [203]:
fill_sentence(xlm,example_r_w_knowledge_3)

'</s> czyli proces, podczas którego substancje zmieniają się z ciekłego stanu w gazowy bez bezpośredniego przechodzenia przez stan stały.'

# Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

In [228]:
rev1="'Ta książka to była rewelacja. Nie mogłem się oderwać od kolejnych stron.'"
rev2="'To miasto to jest koszmar. Nie mogłem się oderwać od myśli o jego chaotycznej strukturze.'"
rev3="'Ten koncert to była niesamowita uczta muzyczna. Nie mogłem się oderwać od dźwięków.'"
rev4="'Ta restauracja to była kulinarna rozczarowanie. Nie mogłem się oderwać od myśli o smaku jedzenia.'"
rev5="'Ten serial to był emocjonalny rollercoaster. Nie mogłem się oderwać od oglądania kolejnych odcinków.'"

In [239]:
ans1=" Wypowiedź ta jest zdecydowanie <mask>."
ans2=" Oceny tego mogą być różne, dla mnie było to <mask>."
ans3=" Poprzednia wypowiedź ma charakter <mask>."

In [257]:
r=0
for x in [rev1,rev2,rev3,rev4,rev5]:
    r+=1
    print(f'\nreview {r}\n')
    p=0
    print("================================")
    for y in [ans1,ans2,ans3]:
        print('-----')
        print(f'prompt: {p}'  + ', model: herbert,  opinion: '+ herbert(x+y)[0]['token_str'])
        print(f'prompt: {p}'  + ', model: xlm       opinion: ' + xlm(x+y)[0]['token_str'])
        print(f'prompt: {p}'  + ', model: bart      opinion: '+ bart(x+y)[0]['token_str'])
        



review 1

-----
prompt: 0, model: herbert,  opinion: prawdziwa
prompt: 0, model: xlm       opinion: najlepsza
prompt: 0, model: bart      opinion: warta
-----
prompt: 0, model: herbert,  opinion: zaskoczenie
prompt: 0, model: xlm       opinion: 10/10
prompt: 0, model: bart      opinion: coś
-----
prompt: 0, model: herbert,  opinion: krytyczny
prompt: 0, model: xlm       opinion: pozytywny
prompt: 0, model: bart      opinion: wyłącznie

review 2

-----
prompt: 0, model: herbert,  opinion: prawdziwa
prompt: 0, model: xlm       opinion: zła
prompt: 0, model: bart      opinion: zbyt
-----
prompt: 0, model: herbert,  opinion: straszne
prompt: 0, model: xlm       opinion: koszmar
prompt: 0, model: bart      opinion: coś
-----
prompt: 0, model: herbert,  opinion: krytyczny
prompt: 0, model: xlm       opinion: historyczny
prompt: 0, model: bart      opinion: wyłącznie

review 3

-----
prompt: 0, model: herbert,  opinion: prawdziwa
prompt: 0, model: xlm       opinion: najlepsza
prompt: 0, mode

# Answer the following questions (2 points):

- Which of the models produced the best results?

Overall the best results were produced by herbert model. He has the most accurate answers when it comes to testing for nominal case. Also model xlm had very good results testing zero-shot learning capabilites. BART model had the worst results, it couldn't even guess word "WODA" in real-world knolwedge sentence.



- Was any of the models able to capture Polish grammar?

As i was saying in the previous question, herbert was the most succesful. Other languages also did good, but not as well as herbert. 

- Was any of the models able to capture long-distant relationships between the words?


herBERT and xlm did very well, they correctly replaced masks, model BERT wasn't so fortunate, it coulnd't guess matching verb.

- Was any of the models able to capture world knowledge?

It was hard task for all models. At least when it came to the second question, herBERT and XLM tried to guess planet name. Model bert couldn't handle this task either.

- Was any of the models good at doing zero-shot classification?

The best job did XLM model. Answers of this model were definitely the most various, in most cases correct. Second best was herBert, which did quite good job guessing the nature of the statement. BERT's answers were mostly without context.

- What are the most striking errors made by the models?


Generating random words by BERT, maybe in the first task, if we gave him possibility to generate more words in one blank, then the results would be better.

XLM couldn't handle simple sentences, for example: Basia nie kupiła <mask>. Results speak for themselves.
    
